# Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.


In [ ]:
import numpy as np
import cv2
import glob
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.patches as patches
%matplotlib inline

In [ ]:
### Helper Functions

def plotImageFiles(filenames):
    for f in filenames:
        fig = plt.figure()
        fig.set_size_inches(3,6)
        img = mpimg.imread(f)
        plt.title(f)
        plt.axis('off')
        plt.imshow(img)
        
def plotImage(img, title=None, cmap=None):
    fig = plt.figure()
    fig.set_size_inches(4,8)
    if title is not None:
        plt.title(title)
    plt.axis('off')
    if cmap is None:
        plt.imshow(img)
    else:
        plt.imshow(img, cmap=cmap)
    
def plotMultipleImages(images, labels=None, ptitle=None, cmap=None):
    """ This function will plot the images specified in a
    single plot.
    """
    numImages = len(images)
    #fig = plt.figure(figsize=(3, 6*numImages))
    fig = plt.figure()
    if ptitle is not None:
        fig.suptitle(ptitle, fontsize="x-large")
    ii = 1
    for img in images:
        ax = fig.add_subplot(1, numImages, ii)
        if labels is not None:
            ax.set_title(labels[ii-1], fontsize="xx-small")
        ax.set_aspect(15)
        plt.axis('off')
        if cmap is not None:
            ax.imshow(img.squeeze(), cmap=cmap)
        else:
            ax.imshow(img.squeeze())
        ii += 1
        
def plotImageAndPolygon(img, polygonPts):
    fig, ax = plt.subplots(1)
    pgn = plt.Polygon(polygonPts, color='r', alpha=0.25, lw=2)
    ax.imshow(img)
    ax.add_patch(pgn)
    
def plotImageAndMultiplePolygons(img, multPolyPts1, multPolyPts2, cmap=None):
    fig, ax = plt.subplots(1)
    if cmap is None:
        ax.imshow(img)
    else:
        ax.imshow(img, cmap=cmap)
    for p in range(0, len(multPolyPts1)):
        pgn1 = plt.Polygon(multPolyPts1[p], color='r', alpha=0.25, lw=1)
        pgn2 = plt.Polygon(multPolyPts2[p], color='b', alpha=0.25, lw=1)
        ax.add_patch(pgn1)
        ax.add_patch(pgn2)
        
def plot3SideBySide(oimg, timg, hist):
    fig = plt.figure(figsize=(12,3))
    plt.subplot(131)
    plt.imshow(oimg)
    plt.subplot(132)
    plt.imshow(timg, cmap="gray")
    plt.subplot(133)
    plt.plot(hist)
    
def plotLaneFit(timg, llfit, rlfit):
    plt.figure()
    # Generate x and y values for plotting
    ploty = np.linspace(0, timg.shape[0]-1, timg.shape[0] )
    left_fitx = llfit[0]*ploty**2 + llfit[1]*ploty + llfit[2]
    right_fitx = rlfit[0]*ploty**2 + rlfit[1]*ploty + rlfit[2]
    plt.imshow(timg, cmap="gray")
    plt.plot(left_fitx, ploty, color='r', lw=3)
    plt.plot(right_fitx, ploty, color='b', lw=3)
    plt.xlim(0, 1280)
    plt.ylim(720, 0)

### Camera Calibration

In [ ]:
### Camera Calibration

def calibrateCamera():
    imfiles = glob.glob('camera_cal/calibration*.jpg')
    nx = 9
    ny = 6
    #3d real world points
    objPoints = []
    #2d camera image points
    imgPoints = []
    #Corners returned by findChessboardCorners contains corners 
    #going left to right and then down. So we have to generate the
    #idealized objpoint to go in the same direction. We also assume
    #that the z-axis is 0 in the objpoint i.e., the image is placed
    #on a 2d plane
    objpoint = np.zeros((nx*ny, 3), np.float32)
    objpoint[:,:2] = np.mgrid[0:nx, 0:ny].T.reshape(-1, 2)
    for imfile in imfiles:
        img = mpimg.imread(imfile)
        #print(imfile)
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        ret, corners = cv2.findChessboardCorners(gray, (nx,ny), None)
        if ret:
            imgPoints.append(corners)
            objPoints.append(objpoint)
    #ret, mtx, dist, rvecs, tvecs
    return cv2.calibrateCamera(objPoints, imgPoints, (gray.shape[1], gray.shape[0]), None, None)
 
def plotUndistortedImages():
    imfiles = glob.glob('camera_cal/calibration*.jpg')
    ret, mtx, dist, rvecs, tvecs = calibrateCamera()
    for imfile in imfiles:
        img = mpimg.imread(imfile)
        dst = cv2.undistort(img, mtx, dist, None, mtx)
        fig = plt.figure()
        fig.add_subplot(1, 2, 1)
        #plt.axis('off')
        plt.imshow(img)
        fig.add_subplot(1, 2, 2)
        plt.imshow(dst)
        
#plotUndistortedImages()
ret, mtx, dist, rvecs, tvecs = calibrateCamera()
print("*** Camera Calibration Complete ***")

### Gradient and Color Thresholding

In [ ]:
def getUndistortedImage(img, _mtx = mtx, _dist = dist):
    return cv2.undistort(img, _mtx, _dist, None, _mtx)

def genericSobelThresholding(img, mfunc, ksize=3, thresh=(0,255)):
    # Grayscale transformation if the image has 3 channels
    # Otherwise assume it has already been transformed
    if img.shape[2] == 3:
        img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Apply cv2.Sobel()
    sobelx = cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize=ksize)
    sobely = cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize=ksize)
    scaled_sobel = mfunc(sobelx, sobely)
    binary_output = np.zeros_like(scaled_sobel)
    binary_output[(scaled_sobel > thresh[0]) & (scaled_sobel < thresh[1])] = 1
    return binary_output

def gradientAbsoluteThresholding(img, orient='x', ksize=3, thresh=(0,255)):
    def gradAbsSobel(sobelx, sobely):
        #Calculate scaled value of gradient
        if orient == 'x':
            sobel = sobelx
        else:
            sobel = sobely
        abs_sobel = np.absolute(sobel)
        scaled_sobel = np.uint8((abs_sobel * 255) / np.max(abs_sobel))
        return scaled_sobel
    return genericSobelThresholding(img, gradAbsSobel, ksize, thresh)

def gradientMagnitudeThresholding(img, ksize=3, thresh=(0,255)):
    def gradMagSobel(sobelx, sobely):
        #Calculate scaled magnitude of gradient
        gradmag = np.sqrt(sobelx**2 + sobely**2)
        scaled_gradmag = np.uint8((gradmag * 255) / np.max(gradmag))
        return scaled_gradmag
    return genericSobelThresholding(img, gradMagSobel, ksize, thresh)

def gradientDirectionThresholding(img, ksize=3, thresh=(0,np.pi/2)):
    def gradDirSobel(sobelx, sobely):
        #Calculate direction of gradient
        absgraddir = np.arctan2(np.absolute(sobely), np.absolute(sobelx))
        return absgraddir
    return genericSobelThresholding(img, gradDirSobel, ksize, thresh)

def colorThresholding(img, channel=0, colspace = cv2.COLOR_RGB2HLS, thresh=(0,255)):
    img = cv2.cvtColor(img, colspace)
    colchannel = img[:,:,channel]
    binary_output = np.zeros_like(colchannel)
    binary_output[(colchannel > thresh[0]) & (colchannel <= thresh[1])] = 1
    return binary_output

print("*** Gradient and Color Thresholding Functions Defined ***")

### Perspective Transformation

In [ ]:
# Compute the perspective transformation matrix using the straight line test images

def getPerspectiveTransformedImage(img, M):
    img_shape = (img.shape[1], img.shape[0])
    warped = cv2.warpPerspective(img, M, img_shape, flags=cv2.INTER_LINEAR)
    return warped

def getPerspectiveTransformationMatrix():
    #Manually identified source points for Image 2
    #bottom-left, bottom-right, top-right, top-left - (x, y)
    ## Other values that also work
    #srcpts2 = np.float32([(337, 633), (975, 633), (709, 463), (575, 463)])
    #dstpts2 = np.float32([(300, 720), (900, 720), (900, 0), (300, 0)])
    srcpts2 = np.float32([(337, 633), (975, 633), (709, 463), (575, 463)])
    dstpts2 = np.float32([(200, 720), (1000, 720), (1000, 0), (200, 0)])
    slfile2 = 'test_images/straight_lines2.jpg'
    M = cv2.getPerspectiveTransform(srcpts2, dstpts2)
    return M

def testPerspectiveTransformation(M):
    #Manually identified source points for Image 1
    #bottom-left, bottom-right, top-right, top-left - (x, y)
    srcpts1 = np.float32([(331, 633), (973, 633), (702, 461), (581, 461)])
    dstpts1 = np.float32([(200, 720), (1000, 720), (1000, 0), (200, 0)])
    slfile1 = 'test_images/straight_lines1.jpg'
    origImg1 = mpimg.imread(slfile1)
    warpedImg1 = getPerspectiveTransformedImage(origImg1, M)
    plotImageAndPolygon(origImg1, srcpts1)
    plotImageAndPolygon(warpedImg1, dstpts1)
    
PTM = getPerspectiveTransformationMatrix()
#testPerspectiveTransformation(PTM)
print("*** Perspective Transformation Complete ***")


### Overall Lane Detection Pipeline

In [ ]:
transformedImages = []
origImages = []

tfiles = glob.glob('test_images/test*.jpg')
for tfile in tfiles:
    img = mpimg.imread(tfile)
    dimg = getUndistortedImage(img)
    colthreshimg_s = colorThresholding(dimg, channel=2, thresh=(180,255))
    absthreshimg = gradientAbsoluteThresholding(dimg, ksize=5, thresh=(20,80))
    gradmagthreshimg = gradientMagnitudeThresholding(dimg, ksize=5, thresh=(30,100))
    #Direction of gradient == +/- np.pi/2 is a vertical line.
    #We'll never have a fully vertical line at this point due to perspective.
    #So use something smaller than np.pi/2 as max
    graddirthreshimg = gradientDirectionThresholding(dimg, ksize=13, thresh=(0.7,1.3))
    combinedimg = np.zeros_like(colthreshimg_s)
    combinedimg[((colthreshimg_s == 1) | (absthreshimg == 1))] = 1
    #combinedimg[((colthreshimg_s == 1) | (absthreshimg == 1)) & (graddirthreshimg == 1)] = 1
    
    #images = [dimg, colthreshimg_s, absthreshimg, gradmagthreshimg, graddirthreshimg, combinedimg]
    #labels = ["Undistorted", "S-Channel Color", "Gradient AbsoluteX", "Gradient Magnitude", "Gradient Direction", "Combined"]
    #plotMultipleImages(images, labels, tfile, "gray")
    #for i in range(0, len(images)):
    #    img = images[i]
    #    lbl = labels[i]
    #    plotImage(img, lbl, cmap="gray")
    origImages.append(dimg)
    transformedImages.append(warpedCombinedImage)
    warpedCombinedImage = getPerspectiveTransformedImage(combinedimg, PTM)
    #plotMultipleImages([dimg, warpedCombinedImage], cmap="gray")

In [ ]:
def slidingWindowLaneDetect(timg, nwindows=9, winextent=100, winshiftthresh=300, enableplot = False):
    """ Returns a tuple that contains the polyfit
    for left lane and right lane respectively
    """
    hist = np.sum(timg[timg.shape[0]/3:, :], axis = 0)
    midpt = hist.shape[0]//2
    llxbase = np.argmax(hist[0:midpt])
    rlxbase = np.argmax(hist[midpt:]) + midpt
    nz = timg.nonzero()
    nzx = np.array(nz[1])
    nzy = np.array(nz[0])    
    #Holds all the non-zero left lane pixels
    ll_pixels = []
    #Holds all the non-zero right lane pixels
    rl_pixels = []
    #Holds window corners for plotting
    llPolyPts = []
    rlPolyPts = []
    for i in range(0, nwindows):
        #Compute left lane sliding window corners' (x,y)
        ll_win_x_left = llxbase - winextent
        ll_win_x_right = llxbase + winextent
        ll_win_y_high = timg.shape[0] - i * (timg.shape[0]//nwindows)
        ll_win_y_low = ll_win_y_high - (timg.shape[0]//nwindows)       
        if enableplot:
            #bl, br, tr, tl
            ll_rect = [(ll_win_x_left, ll_win_y_low), (ll_win_x_right, ll_win_y_low),\
                       (ll_win_x_right, ll_win_y_high), (ll_win_x_left, ll_win_y_high)]
            llPolyPts.append(ll_rect)
        #Compute right lane sliding window corners' (x,y)
        rl_win_x_left = rlxbase - winextent
        rl_win_x_right = rlxbase + winextent
        rl_win_y_high = timg.shape[0] - i * (timg.shape[0]//nwindows)
        rl_win_y_low = rl_win_y_high - (timg.shape[0]//nwindows)
        if enableplot:
            #bl, br, tr, tl
            rl_rect = [(rl_win_x_left, rl_win_y_low), (rl_win_x_right, rl_win_y_low),\
                       (rl_win_x_right, rl_win_y_high), (rl_win_x_left, rl_win_y_high)]
            rlPolyPts.append(rl_rect)
        #Compute pixels that are non-zero within sliding window
        ll_win_pixels = ((nzx >= ll_win_x_left) & (nzx < ll_win_x_right)\
                                 & (nzy >= ll_win_y_low) & (nzy < ll_win_y_high)).nonzero()[0]
        ll_pixels.append(ll_win_pixels)
        
        rl_win_pixels = ((nzx >= rl_win_x_left) & (nzx < rl_win_x_right)\
                                 & (nzy >= rl_win_y_low) & (nzy < rl_win_y_high)).nonzero()[0]
        rl_pixels.append(rl_win_pixels)
        
        #Shift lane bases if we have a lot of non-zero pixels in this sliding window
        if len(ll_win_pixels) > winshiftthresh:
            llxbase = int(np.mean(nzx[ll_win_pixels]))
        if len(rl_win_pixels) > winshiftthresh:
            rlxbase = int(np.mean(nzx[rl_win_pixels]))
    
    if enableplot:
        plotImageAndMultiplePolygons(timg, llPolyPts, rlPolyPts, cmap="gray")
    
    ll_pixels = np.concatenate(ll_pixels)
    rl_pixels = np.concatenate(rl_pixels)
    llx = nzx[ll_pixels]
    lly = nzy[ll_pixels]
    rlx = nzx[rl_pixels]
    rly = nzy[rl_pixels]
    
    llfit = np.polyfit(lly, llx, 2)
    rlfit = np.polyfit(rly, rlx, 2)
    
    return llfit, rlfit
    
for i in range(0, len(transformedImages)):
    oimg = origImages[i]
    timg = transformedImages[i]
    #hist = np.sum(timg[timg.shape[0]/3:, :], axis = 0)
    #plot3SideBySide(oimg, timg, hist)
    llfit, rlfit = slidingWindowLaneDetect(timg)
    plotLaneFit(timg, llfit, rlfit)
    